In [1]:
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
import re
import joblib
import pickle

In [2]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [3]:
def text_cleaner(content):
    newString = content.lower()
    newString = BeautifulSoup(newString, "html.parser").get_text()
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    newString = re.sub('[m]{2,}', 'mm', newString)
    return newString

In [4]:
text = "I feel very much depressed. I feel suicidal"
clean_text = text_cleaner(text)
clean_text

'i feel very much depressed  i feel suicidal'

In [62]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [63]:
df = pd.read_excel("dataset.xlsx")
df = df.dropna()

In [64]:
df['cleaned'] = df["text"].apply(text_cleaner)

C:\Users\User\AppData\Local\Temp\ipykernel_27600\2531886863.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  newString = BeautifulSoup(newString, "html.parser").get_text()


In [84]:
X = df.cleaned
y = df.label

In [85]:
type(X[0])


str

In [86]:
X.shape, y.shape

((6970,), (6970,))

In [89]:
vect = CountVectorizer(max_features = 20000 , lowercase=False , ngram_range=(1,2))
X_cv =vect.fit_transform(X).toarray()

In [101]:
vect.vocabulary_

{'oh': 8901,
 'my': 6291,
 'gosh': 3712,
 'oh my': 8920,
 'my gosh': 6338,
 'trouble': 17078,
 'sleeping': 15025,
 'confused': 2115,
 'mind': 6009,
 'restless': 14240,
 'heart': 4070,
 'all': 278,
 'out': 10122,
 'of': 8281,
 'trouble sleeping': 17079,
 'restless heart': 14284,
 'all out': 294,
 'out of': 10166,
 'of tune': 8733,
 'wrong': 18593,
 'back': 1032,
 'off': 8774,
 'dear': 2371,
 'forward': 3378,
 'doubt': 2595,
 'stay': 15311,
 'in': 4476,
 'and': 495,
 'place': 11510,
 'wrong back': 18601,
 'back off': 1037,
 'off dear': 8782,
 'stay in': 15315,
 'restless and': 14249,
 'and restless': 580,
 'restless place': 14318,
 'have': 3957,
 'focus': 3227,
 'to': 16699,
 'something': 15179,
 'else': 2767,
 'but': 1569,
 'am': 382,
 'still': 15324,
 'worried': 18388,
 'something else': 15183,
 'else but': 2768,
 'but am': 1574,
 'am still': 443,
 'it': 4877,
 'is': 4627,
 'been': 1265,
 'month': 6089,
 'now': 8038,
 'boy': 1484,
 'what': 17710,
 'do': 2529,
 'you': 19202,
 'mean': 59

In [94]:
a = vect.transform([clean_text]).toarray()

for i in a[0]:
    if i != 0:
        print(i)

1
2
1
1
1
1


In [5]:
loaded_model = pickle.load(open('CV_BestModel.sav', 'rb'))
single_prediction = loaded_model.predict(vect.transform([clean_text]).toarray())[0]

NameError: name 'vect' is not defined

In [6]:
loaded_model

AdaBoostClassifier()

In [99]:
single_prediction[0]

0.0